---
execute:
  cache: false
  eval: true
  echo: true
  warning: false
---

# Sequential Parameter Optimization: Using `scipy` Optimizers {#sec-scipy-optimizers}

As a default optimizer, `spotpython` uses `differential_evolution` from the `scipy.optimize` package. Alternatively, any other optimizer from the `scipy.optimize` package can be used. This chapter describes how different optimizers from the `scipy optimize` package can be used on the surrogate.
The optimization algorithms are available from [https://docs.scipy.org/doc/scipy/reference/optimize.html](https://docs.scipy.org/doc/scipy/reference/optimize.html)

In [1]:
#| label: 04_imports
import numpy as np
from math import inf
from spotpython.fun.objectivefunctions import analytical
from spotpython.spot import spot
from scipy.optimize import shgo
from scipy.optimize import direct
from scipy.optimize import differential_evolution
from scipy.optimize import dual_annealing
from scipy.optimize import basinhopping
from spotpython.utils.init import fun_control_init, design_control_init, optimizer_control_init, surrogate_control_init

Seed set to 123


## The Objective Function Branin

The `spotpython` package provides several classes of objective functions. We will use an analytical objective function, i.e., a function that can be described by a (closed) formula. Here we will use the Branin function. The 2-dim Branin function is
$$
y = a  (x_2 - b  x_1^2 + c  x_1 - r) ^2 + s  (1 - t)  \cos(x_1) + s,
$$
where values of $a$, $b$, $c$, $r$, $s$ and $t$ are: $a = 1$, $b = 5.1 / (4\pi^2)$, $c = 5 / \pi$, $r = 6$, $s = 10$ and $t = 1 / (8\pi)$.

It has three global minima: $f(x) = 0.397887$ at $(-\pi, 12.275)$, $(\pi, 2.275)$, and $(9.42478, 2.475)$.

Input Domain: This function is usually evaluated on the square  $x_1 \in  [-5, 10] \times x_2 \in  [0, 15]$.

In [2]:
#| label: 04_objective_function
from spotpython.fun.objectivefunctions import analytical
lower = np.array([-5,-0])
upper = np.array([10,15])
fun = analytical(seed=123).fun_branin

## The Optimizer{#sec-optimizer}

Differential Evolution (DE) from the `scikit.optimize` package, see [https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.differential_evolution.html#scipy.optimize.differential_evolution](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.differential_evolution.html#scipy.optimize.differential_evolution) is the default optimizer for the search on the surrogate.
Other optimiers that are available in `spotpython`, see [https://docs.scipy.org/doc/scipy/reference/optimize.html#global-optimization](https://docs.scipy.org/doc/scipy/reference/optimize.html#global-optimization).

  * `dual_annealing`
  * `direct`
  * `shgo`
  * `basinhopping`

These optimizers can be selected as follows:

In [3]:
#| label: 04_optimizer_control
#| eval: false
from scipy.optimize import differential_evolution
optimizer = differential_evolution

As noted above, we will use `differential_evolution`. The optimizer can use `1000` evaluations. This value will be passed to the `differential_evolution` method, which has the argument `maxiter` (int). It defines the maximum number of generations over which the entire differential evolution population is evolved, see [https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.differential_evolution.html#scipy.optimize.differential_evolution](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.differential_evolution.html#scipy.optimize.differential_evolution)


:::{.callout-note}
#### TensorBoard

Similar to the one-dimensional case, which is discussed in @sec-visualizing-tensorboard-01, we can use TensorBoard to monitor the progress of the optimization. We will use a similar code, only the prefix is different:

In [4]:
fun_control=fun_control_init(
                    lower = lower,
                    upper = upper,
                    fun_evals = 20,
                    PREFIX = "04_DE_"
                    )
surrogate_control=surrogate_control_init(
                    n_theta=len(lower))

Seed set to 123


:::

In [5]:
spot_de = spot.Spot(fun=fun,
                    fun_control=fun_control,
                    surrogate_control=surrogate_control)
spot_de.run()

spotpython tuning: 3.8004550038787155 [######----] 55.00% 
spotpython tuning: 3.8004550038787155 [######----] 60.00% 
spotpython tuning: 3.1588579885698627 [######----] 65.00% 
spotpython tuning: 3.1342382932317037 [#######---] 70.00% 
spotpython tuning: 2.8956615907630585 [########--] 75.00% 
spotpython tuning: 0.42052429574482275 [########--] 80.00% 
spotpython tuning: 0.4013351867835322 [########--] 85.00% 
spotpython tuning: 0.399265616254338 [#########-] 90.00% 
spotpython tuning: 0.399265616254338 [##########] 95.00% 
spotpython tuning: 0.399265616254338 [##########] 100.00% Done...



### TensorBoard

If the `prefix` argument in `fun_control_init()`is not `None` (as above, where the `prefix` was set to `04_DE_`) , we can start TensorBoard in the background with the following command:

```{raw}
tensorboard --logdir="./runs"
```

We can access the TensorBoard web server with the following URL:

```{raw}
http://localhost:6006/
```

The TensorBoard plot illustrates how `spotpython` can be used as a microscope for the internal mechanisms of the surrogate-based optimization process. Here, one important parameter, the learning rate $\theta$ of the Kriging surrogate is plotted against the number of optimization steps.

![TensorBoard visualization of the spotpython optimization process and the surrogate model.](figures_static/05_tensorboard_01.png){width="100%"}


## Print the Results

In [6]:
spot_de.print_results()

min y: 0.399265616254338
x0: 3.151170754781285
x1: 2.2981660114765448


[['x0', 3.151170754781285], ['x1', 2.2981660114765448]]

## Show the Progress

In [7]:
spot_de.plot_progress(log_y=True)

In [8]:
spot_de.surrogate.plot()

invalid command name "14359732672process_stream_events"
    while executing
"14359732672process_stream_events"
    ("after" script)
can't invoke "event" command: application has been destroyed
    while executing
"event generate $w <<ThemeChanged>>"
    (procedure "ttk::ThemeChanged" line 6)
    invoked from within
"ttk::ThemeChanged"


## Exercises


### `dual_annealing`

* Describe the optimization algorithm, see [scipy.optimize.dual_annealing](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.dual_annealing.html).
* Use the algorithm as an optimizer on the surrogate.

:::{.callout-tip}
##### Tip: Selecting the Optimizer for the Surrogate

We can run spotpython with the `dual_annealing` optimizer as follows:

In [9]:
spot_da = spot.Spot(fun=fun,
                    fun_control=fun_control,
                    optimizer=dual_annealing,
                    surrogate_control=surrogate_control)
spot_da.run()
spot_da.print_results()
spot_da.plot_progress(log_y=True)
spot_da.surrogate.plot()

spotpython tuning: 3.8004480172281534 [######----] 55.00% 
spotpython tuning: 3.8004480172281534 [######----] 60.00% 
spotpython tuning: 3.158996247273234 [######----] 65.00% 
spotpython tuning: 3.134218255713952 [#######---] 70.00% 
spotpython tuning: 2.8926591957342467 [########--] 75.00% 
spotpython tuning: 0.4189006494820333 [########--] 80.00% 
spotpython tuning: 0.4019392204560983 [########--] 85.00% 
spotpython tuning: 0.39922543271904765 [#########-] 90.00% 
spotpython tuning: 0.39922543271904765 [##########] 95.00% 
spotpython tuning: 0.39922543271904765 [##########] 100.00% Done...

min y: 0.39922543271904765
x0: 3.1506699177492252
x1: 2.298631597428197


invalid command name "14359227776process_stream_events"
    while executing
"14359227776process_stream_events"
    ("after" script)
can't invoke "event" command: application has been destroyed
    while executing
"event generate $w <<ThemeChanged>>"
    (procedure "ttk::ThemeChanged" line 6)
    invoked from within
"ttk::ThemeChanged"


:::


### `direct`

* Describe the optimization algorithm
* Use the algorithm as an optimizer on the surrogate

:::{.callout-tip}
##### Tip: Selecting the Optimizer for the Surrogate

We can run spotpython with the `direct` optimizer as follows:

In [10]:
spot_di = spot.Spot(fun=fun,
                    fun_control=fun_control,
                    optimizer=direct,
                    surrogate_control=surrogate_control)
spot_di.run()
spot_di.print_results()
spot_di.plot_progress(log_y=True)
spot_di.surrogate.plot()

spotpython tuning: 3.812970247994418 [######----] 55.00% 
spotpython tuning: 3.812970247994418 [######----] 60.00% 
spotpython tuning: 3.162514679816068 [######----] 65.00% 
spotpython tuning: 3.1189615135325983 [#######---] 70.00% 
spotpython tuning: 2.6597698275013038 [########--] 75.00% 
spotpython tuning: 0.3984917773445744 [########--] 80.00% 
spotpython tuning: 0.3984917773445744 [########--] 85.00% 
spotpython tuning: 0.3984917773445744 [#########-] 90.00% 
spotpython tuning: 0.3984917773445744 [##########] 95.00% 
spotpython tuning: 0.3984917773445744 [##########] 100.00% Done...

min y: 0.3984917773445744
x0: 3.137860082304525
x1: 2.3010973936899863


invalid command name "14445919232process_stream_events"
    while executing
"14445919232process_stream_events"
    ("after" script)
can't invoke "event" command: application has been destroyed
    while executing
"event generate $w <<ThemeChanged>>"
    (procedure "ttk::ThemeChanged" line 6)
    invoked from within
"ttk::ThemeChanged"


:::

### `shgo`

* Describe the optimization algorithm
* Use the algorithm as an optimizer on the surrogate

:::{.callout-tip}
##### Tip: Selecting the Optimizer for the Surrogate

We can run spotpython with the `direct` optimizer as follows:

In [11]:
spot_sh = spot.Spot(fun=fun,
                    fun_control=fun_control,
                    optimizer=shgo,
                    surrogate_control=surrogate_control)
spot_sh.run()
spot_sh.print_results()
spot_sh.plot_progress(log_y=True)
spot_sh.surrogate.plot()

spotpython tuning: 3.8004562736456844 [######----] 55.00% 
spotpython tuning: 3.8004562736456844 [######----] 60.00% 
spotpython tuning: 3.158996879015902 [######----] 65.00% 
spotpython tuning: 3.1341298968229996 [#######---] 70.00% 
spotpython tuning: 2.8919915800445954 [########--] 75.00% 
spotpython tuning: 0.4173165753511867 [########--] 80.00% 
spotpython tuning: 0.40097732409794773 [########--] 85.00% 
spotpython tuning: 0.3993020098909934 [#########-] 90.00% 
spotpython tuning: 0.3993020098909934 [##########] 95.00% 
spotpython tuning: 0.3993020098909934 [##########] 100.00% Done...

min y: 0.3993020098909934
x0: 3.1510894339439672
x1: 2.298936853041466


invalid command name "14424901248process_stream_events"
    while executing
"14424901248process_stream_events"
    ("after" script)
can't invoke "event" command: application has been destroyed
    while executing
"event generate $w <<ThemeChanged>>"
    (procedure "ttk::ThemeChanged" line 6)
    invoked from within
"ttk::ThemeChanged"


:::



### `basinhopping`

* Describe the optimization algorithm
* Use the algorithm as an optimizer on the surrogate

:::{.callout-tip}
##### Tip: Selecting the Optimizer for the Surrogate

We can run spotpython with the `direct` optimizer as follows:

In [12]:
spot_bh = spot.Spot(fun=fun,
                    fun_control=fun_control,
                    optimizer=basinhopping,
                    surrogate_control=surrogate_control)
spot_bh.run()
spot_bh.print_results()
spot_bh.plot_progress(log_y=True)
spot_bh.surrogate.plot()

spotpython tuning: 3.800453600053931 [######----] 55.00% 
spotpython tuning: 3.800453600053931 [######----] 60.00% 
spotpython tuning: 3.1590141837294237 [######----] 65.00% 
spotpython tuning: 3.1341341806066314 [#######---] 70.00% 
spotpython tuning: 2.8914331943522242 [########--] 75.00% 
spotpython tuning: 0.41214245125719984 [########--] 80.00% 
spotpython tuning: 0.40113843843078634 [########--] 85.00% 
spotpython tuning: 0.3992327747775164 [#########-] 90.00% 
spotpython tuning: 0.3992327747775164 [##########] 95.00% 
spotpython tuning: 0.3992327747775164 [##########] 100.00% Done...

min y: 0.3992327747775164
x0: 3.15016404734246
x1: 2.2998320162156896


invalid command name "14408781376process_stream_events"
    while executing
"14408781376process_stream_events"
    ("after" script)
can't invoke "event" command: application has been destroyed
    while executing
"event generate $w <<ThemeChanged>>"
    (procedure "ttk::ThemeChanged" line 6)
    invoked from within
"ttk::ThemeChanged"


:::


### Performance Comparison

Compare the performance and run time of the 5 different optimizers:

  * `differential_evolution`
  * `dual_annealing`
  *  `direct`
  * `shgo`
  * `basinhopping`.

The Branin function has three global minima:

* $f(x) = 0.397887$  at 
  * $(-\pi, 12.275)$, 
  * $(\pi, 2.275)$, and 
  * $(9.42478, 2.475)$.    
* Which optima are found by the optimizers?
* Does the `seed` argument in `fun = analytical(seed=123).fun_branin` change this behavior?

## Jupyter Notebook

:::{.callout-note}

* The Jupyter-Notebook of this chapter is available on GitHub in the [Hyperparameter-Tuning-Cookbook Repository](https://github.com/sequential-parameter-optimization/Hyperparameter-Tuning-Cookbook/blob/main/004_spot_sklearn_optimization.ipynb)

:::